In [1]:
# --- Notebook Cell 1: Setup & helpers ---

import asyncio
import json
import os

os.environ.setdefault("GRPC_VERBOSITY", "NONE")
os.environ.setdefault("GLOG_minloglevel", "2")
from src.pipeline_4_agent import SequentialAgentPipeline
from src.retrieval_then_llm import SmartClassifier
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import pandas as pd
from difflib import SequenceMatcher

# Your project imports
# from your_module import SequentialAgentPipeline, SmartClassifier

# ---------- Config ----------
DATA_DIR = "data"
TICKETS_CSV = os.path.join(DATA_DIR, "dataset_for_categorization.csv")  # input
TAXONOMY_CSV = os.path.join(DATA_DIR, "cleaned.csv")  # taxonomy
OUT_DETAILED = os.path.join(DATA_DIR, "evaluated_detailed.csv")  # output (rows)
OUT_SUMMARY = os.path.join(DATA_DIR, "metrics_summary.csv")  # output (summary)

# ---------- Ground truth column fallbacks ----------
GT_CANDIDATE_NAMES = [
    # (L1, L2, L3, L4) ordered by preference; we pick the first set that exists
    ("gt_domain", "gt_category_1", "gt_category_2", "gt_category_3"),
    ("truth_domain", "truth_cat1", "truth_cat2", "truth_cat3"),
    ("Domain", "Cat1", "Cat2", "Cat3"),
    ("domain", "category_1", "category_2", "category_3"),
]

# ---------- Utilities ----------


def first_existing_column_set(
    df: pd.DataFrame, candidates: List[Tuple[str, str, str, str]]
) -> Optional[Tuple[str, str, str, str]]:
    for cols in candidates:
        if all(c in df.columns for c in cols):
            return cols
    return None


def normalize_str(x: Optional[str]) -> str:
    if x is None:
        return ""
    return str(x).strip()


def build_path(*parts: str) -> str:
    parts = [normalize_str(p) for p in parts if normalize_str(p)]
    return " > ".join(parts)


def seq_extract_names(seq_dict: Dict[str, str]) -> Tuple[str, str, str, str]:
    """
    SequentialAgentPipeline output (your example):
      keys_in_order = ["domain", "category_1", "category_2", "category_3"]
    Missing keys or blanks are normalized to "".
    """
    keys = ["domain", "category_1", "category_2", "category_3"]
    vals = [
        normalize_str(seq_dict.get(k)) if isinstance(seq_dict, dict) else ""
        for k in keys
    ]
    return tuple(vals)  # (L1, L2, L3, L4)


def kb_extract_levels(kb_obj: Dict) -> Dict[str, Dict[str, Optional[str]]]:
    """
    SmartClassifier output (your example):
      {
        "best_path": {
          "L1_domain": {"id": "...", "confidence": 0.9, "name": "..."},
          ...
        },
        "selected_index": 42,
        "abstain": False,
        "rationale": "...",
        "path_mn": "..."
      }
    """
    best = kb_obj.get("best_path", {}) or {}

    def get_node(key: str) -> Dict[str, Optional[str]]:
        node = best.get(key, {}) or {}
        return {
            "id": node.get("id"),
            "name": node.get("name"),
            "confidence": node.get("confidence"),
        }

    return {
        "L1": get_node("L1_domain"),
        "L2": get_node("L2_cat1"),
        "L3": get_node("L3_cat2"),
        "L4": get_node("L4_cat3"),
        "meta": {
            "selected_index": kb_obj.get("selected_index"),
            "abstain": kb_obj.get("abstain"),
            "rationale": kb_obj.get("rationale"),
            "path_mn": kb_obj.get("path_mn"),
        },
    }


def fuzzy_ratio(a: str, b: str) -> float:
    # simple 0..1 using difflib (token order preserved; good enough without extra deps)
    return SequenceMatcher(None, normalize_str(a), normalize_str(b)).ratio()


def part_match_score(
    gt: Tuple[str, str, str, str], pred: Tuple[str, str, str, str]
) -> float:
    """
    Depth-aware prefix match.
    Score = hits/depth with steps in {0,.25,.5,.75,1} (when depth=4); fewer steps with shallower GT.
    """
    gt_levels = [normalize_str(x) for x in gt]
    pr_levels = [normalize_str(x) for x in pred]
    # how many GT levels actually exist (non-empty)
    depth = 0
    for x in gt_levels:
        if x:
            depth += 1
        else:
            break
    if depth == 0:
        return 0.0  # no GT available

    hits = 0
    for i in range(depth):
        if gt_levels[i] and (gt_levels[i] == pr_levels[i]):
            hits += 1
        else:
            break
    return hits / depth

In [2]:
# --- Notebook Cell 2: Run classification & build evaluation table ---

import asyncio
from tqdm import tqdm  # optional; if unavailable, comment out

# Instantiate your systems
pipeline = SequentialAgentPipeline(TAXONOMY_CSV)
classifier = SmartClassifier(csv_path=TAXONOMY_CSV)

# Load tickets
tickets = pd.read_csv(TICKETS_CSV, encoding="utf-8-sig")

# Try to locate ground-truth columns
gt_cols = first_existing_column_set(tickets, GT_CANDIDATE_NAMES)
if gt_cols is None:
    # Create blank GT columns if none found (keeps the pipeline working)
    gt_cols = ("gt_domain", "gt_category_1", "gt_category_2", "gt_category_3")
    for c in gt_cols:
        if c not in tickets.columns:
            tickets[c] = ""

GT_L1, GT_L2, GT_L3, GT_L4 = gt_cols

# Prepare containers
rows = []

# Async concurrency limiter (avoid hammering LLMs)
SEMAPHORE = asyncio.Semaphore(4)


async def process_ticket(ticket_text: str):
    async with SEMAPHORE:
        # sequential path
        seq_pred = await pipeline.run(ticket_text)
        seq_L1, seq_L2, seq_L3, seq_L4 = seq_extract_names(seq_pred)
        seq_path = build_path(seq_L1, seq_L2, seq_L3, seq_L4)

        # KB path
        kb_obj = await classifier.classify(ticket_text)
        kb = kb_extract_levels(kb_obj)
        kb_L1 = normalize_str(kb["L1"]["name"])
        kb_L2 = normalize_str(kb["L2"]["name"])
        kb_L3 = normalize_str(kb["L3"]["name"])
        kb_L4 = normalize_str(kb["L4"]["name"])
        kb_path = normalize_str(kb["meta"]["path_mn"]) or build_path(
            kb_L1, kb_L2, kb_L3, kb_L4
        )

        return {
            "seq": {
                "L1": seq_L1,
                "L2": seq_L2,
                "L3": seq_L3,
                "L4": seq_L4,
                "path": seq_path,
            },
            "kb": {
                "L1_name": kb_L1,
                "L2_name": kb_L2,
                "L3_name": kb_L3,
                "L4_name": kb_L4,
                "L1_id": normalize_str(kb["L1"]["id"]),
                "L2_id": normalize_str(kb["L2"]["id"]),
                "L3_id": normalize_str(kb["L3"]["id"]),
                "L4_id": normalize_str(kb["L4"]["id"]),
                "L1_conf": kb["L1"]["confidence"],
                "L2_conf": kb["L2"]["confidence"],
                "L3_conf": kb["L3"]["confidence"],
                "L4_conf": kb["L4"]["confidence"],
                "selected_index": kb["meta"]["selected_index"],
                "abstain": kb["meta"]["abstain"],
                "rationale": kb["meta"]["rationale"],
                "path": kb_path,
            },
            "raw": kb_obj,  # optional/debug
        }


async def run_all():
    tasks = []
    for txt in tickets["ticket"].astype(str).tolist():
        tasks.append(asyncio.create_task(process_ticket(txt)))
    out = []
    for t in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Classifying"):
        out.append(await t)
    # preserve original order
    return [r for r in out]


# --- Execute the async pipeline in both notebook & script contexts ---

import sys, asyncio

async def _run_all_wrapper():
    return await run_all()

IN_IPYTHON = "ipykernel" in sys.modules or "IPython" in sys.modules

if IN_IPYTHON:
    # Notebook/Colab: just await (top-level await is supported)
    results = await _run_all_wrapper()
else:
    # Plain Python script
    results = asyncio.run(_run_all_wrapper())
# Build a row-wise dataframe with GT, seq, kb, and metrics
for i, res in enumerate(results):
    row_src = tickets.iloc[i]
    gt_tuple = (
        normalize_str(row_src[GT_L1]),
        normalize_str(row_src[GT_L2]),
        normalize_str(row_src[GT_L3]),
        normalize_str(row_src[GT_L4]),
    )

    seq_tuple = (res["seq"]["L1"], res["seq"]["L2"], res["seq"]["L3"], res["seq"]["L4"])
    kb_tuple = (
        res["kb"]["L1_name"],
        res["kb"]["L2_name"],
        res["kb"]["L3_name"],
        res["kb"]["L4_name"],
    )

    # Metrics
    pm_seq = part_match_score(gt_tuple, seq_tuple)
    pm_kb = part_match_score(gt_tuple, kb_tuple)
    fz_seq = fuzzy_ratio(build_path(*gt_tuple), res["seq"]["path"])
    fz_kb = fuzzy_ratio(build_path(*gt_tuple), res["kb"]["path"])

    rows.append(
        {
            # raw ticket text
            "ticket": row_src.get("ticket", ""),
            # ground truth (detected columns)
            "gt_domain": gt_tuple[0],
            "gt_category_1": gt_tuple[1],
            "gt_category_2": gt_tuple[2],
            "gt_category_3": gt_tuple[3],
            "gt_path": build_path(*gt_tuple),
            # sequential agent preds (names only)
            "seq_domain": res["seq"]["L1"],
            "seq_category_1": res["seq"]["L2"],
            "seq_category_2": res["seq"]["L3"],
            "seq_category_3": res["seq"]["L4"],
            "seq_path": res["seq"]["path"],
            # kb agent preds (names + ids + confidences)
            "kb_domain": res["kb"]["L1_name"],
            "kb_category_1": res["kb"]["L2_name"],
            "kb_category_2": res["kb"]["L3_name"],
            "kb_category_3": res["kb"]["L4_name"],
            "kb_domain_id": res["kb"]["L1_id"],
            "kb_category_1_id": res["kb"]["L2_id"],
            "kb_category_2_id": res["kb"]["L3_id"],
            "kb_category_3_id": res["kb"]["L4_id"],
            "kb_conf_L1": res["kb"]["L1_conf"],
            "kb_conf_L2": res["kb"]["L2_conf"],
            "kb_conf_L3": res["kb"]["L3_conf"],
            "kb_conf_L4": res["kb"]["L4_conf"],
            "kb_selected_index": res["kb"]["selected_index"],
            "kb_abstain": res["kb"]["abstain"],
            "kb_rationale": res["kb"]["rationale"],
            "kb_path": res["kb"]["path"],
            # metrics
            "PartMatch_seq": pm_seq,
            "PartMatch_kb": pm_kb,
            "Fuzzy_seq": fz_seq,
            "Fuzzy_kb": fz_kb,
        }
    )

evaluated = pd.DataFrame(rows)

# Persist detailed rows
os.makedirs(DATA_DIR, exist_ok=True)
evaluated.to_csv(OUT_DETAILED, index=False, encoding="utf-8-sig")

# Produce a compact summary
summary = (
    evaluated[["PartMatch_seq", "PartMatch_kb", "Fuzzy_seq", "Fuzzy_kb"]]
    .agg(["mean"])
    .rename(index={"mean": "average"})
)
summary.to_csv(OUT_SUMMARY, encoding="utf-8-sig")
evaluated.head(3)

Loading and preparing taxonomy from data/cleaned.csv...
Initializing Smart Classifier (Domain→Cat1→Cat2→Cat3)...
Loading existing FAISS index from taxonomy.index...


Classifying:   0%|          | 0/46 [00:00<?, ?it/s]

Toki>Үйлчилгээ>Гар утас>Апп-тай холбоотой
Path terminates at Cat2 for 'U-Point -> Гомдол -> Апп-тай холбоотой'.
Toki>Бусад>Physical сувагтай холбоотой>L2-Хэрэглэгчид мэдээлэл буруу дутуу хүргэснээс шалтгаалсан
Toki>Хэтэвч>Данс цэнэглэлттэй холбоотой>L2-Failed transactions гүйлгээ орж ирээгүй


Classifying:   9%|▊         | 4/46 [00:11<01:23,  2.00s/it]

Toki>Хэтэвч>Кредит эрх тооцоолохтой холбоотой>L2-Зээлийн эрх тооцоологдоогүй
Toki>Хэтэвч>Кредит эргэн төлөлттэй холбоотой>L2-Эргэн төлөлт оруулсан


...

Ger Internet>Сүлжээ>Интернэт орохгүй, хурд удаан байгаа>L2-Инженер шалгахад хэвийн байсан


Classifying:  96%|█████████▌| 44/46 [01:37<00:03,  1.72s/it]

Path terminates at Cat2 for 'Ger Internet -> Интернэтийн нэр, нууц үгтэй холбоотой -> Wifi нууц үг сольсон'.
Ger Internet>Дата>Дата уншихгүй, хурд удаан байгаа>L2-Дата утга зөрснөөс шалтгаалсан


Classifying: 100%|██████████| 46/46 [01:41<00:00,  2.21s/it]


,ticket,gt_domain,gt_category_1,gt_category_2,gt_category_3,gt_path,seq_domain,seq_category_1,seq_category_2,seq_category_3,...,kb_conf_L3,kb_conf_L4,kb_selected_index,kb_abstain,kb_rationale,kb_path,PartMatch_seq,PartMatch_kb,Fuzzy_seq,Fuzzy_kb
0,"Эрдэнэболд Эрдэнэбилэг': ""ажилтантай холбогдох...",,,,,,Toki,Гомдол,Physical сувагтай холбоотой,Апп-тай холбоотой,...,1.0,1.0,9,False,Сонгосон зам: Toki > Хэтэвч > Paylater үйлчилг...,Toki > Хэтэвч > Paylater үйлчилгээ > L2-Эргэн ...,0.0,0.0,0.0,0.0
1,"customer service': ""Та төлбөрийн гүйлгээний ху...",,,,,,Toki,Гомдол,Апп-тай холбоотой,L2-Failed transactions гүйлгээ орж ирээгүй,...,1.0,1.0,8,False,Сонгосон зам: Toki > Хэтэвч > Данс цэнэглэлттэ...,Toki > Хэтэвч > Данс цэнэглэлттэй холбоотой > ...,0.0,0.0,0.0,0.0
2,"Claria Ryy': ""Tgvel toki zeeliin erheere oor g...",,,,,,Toki,Гомдол,Апп-тай холбоотой,L2-Хэрэглэгчид мэдээлэл буруу дутуу хүргэснээс...,...,0.0,0.0,19,True,Сонгосон зам: Toki > Үйлчилгээ > Дэлгүүрүүд > ...,"Toki > Үйлчилгээ > Дэлгүүрүүд > Захиалга, хүрг...",0.0,0.0,0.0,0.0


In [3]:
# --- Notebook Cell 3: Optional utilities for threshold sweeps ---


def summarize_by_system(df: pd.DataFrame) -> pd.DataFrame:
    out = []
    out.append(
        {
            "system": "sequential",
            "PartMatch": df["PartMatch_seq"].mean(),
            "Fuzzy": df["Fuzzy_seq"].mean(),
        }
    )
    out.append(
        {
            "system": "kb agent",
            "PartMatch": df["PartMatch_kb"].mean(),
            "Fuzzy": df["Fuzzy_kb"].mean(),
        }
    )
    return pd.DataFrame(out).sort_values(["PartMatch", "Fuzzy"], ascending=False)


display(summarize_by_system(evaluated))

,system,PartMatch,Fuzzy
0,sequential,0.0,0.0
1,kb agent,0.0,0.0
